# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,east london,94,ZA,1588544827,76,-33.02,27.91,66.20,11.23
1,1,albany,40,US,1588544824,27,42.60,-73.97,77.00,9.17
2,2,pangody,90,RU,1588545028,96,65.85,74.49,17.13,14.47
3,3,ponta do sol,20,PT,1588544832,77,32.67,-17.10,66.20,23.04
4,4,butaritari,100,KI,1588544833,82,3.07,172.79,81.70,10.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = city_data_df[["Lat", "Lng"]]

weight = city_data_df["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Filter vacation temps
vacation_weather = city_data_df[(city_data_df["Max Temperature"] > 70) & (city_data_df["Max Temperature"] < 80)  & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)].dropna()
vacation_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
55,55,namibe,0,AO,1588544829,56,-15.20,12.15,78.37,1.34
358,358,kapoeta,0,SS,1588545080,52,4.77,33.59,79.29,4.72
404,404,lobito,0,AO,1588545086,83,-12.35,13.55,77.83,1.92
522,522,shenjiamen,0,CN,1588545103,74,29.96,122.30,71.78,5.48
524,524,riyadh,0,SA,1588545011,29,24.69,46.72,77.00,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = vacation_weather[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
55,namibe,AO,-15.20,12.15,
358,kapoeta,SS,4.77,33.59,
404,lobito,AO,-12.35,13.55,
522,shenjiamen,CN,29.96,122.30,
524,riyadh,SA,24.69,46.72,


In [53]:
hotel_name = []
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [50]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()

,City,Country,Lat,Lng,Hotel Name
55,namibe,AO,-15.20,12.15,Chik-Chik Namibe
358,kapoeta,SS,4.77,33.59,Eastgate Hotel
404,lobito,AO,-12.35,13.55,Casa Rosa Hotel Residence
522,shenjiamen,CN,29.96,122.30,Jinjiang Inn Zhoushan Putuo Shenjiamen
524,riyadh,SA,24.69,46.72,InterContinental Riyadh


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))